In [80]:
import notebook_common
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    r_word_2400_xlsx,
    w_word_2400_csv,
    w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    # w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    pattern_table_spoken,
    pattern_td_spoken,
    r_spoken_freq_nlb_with_enter_txt,
    w_spoken_freq_nlb_with_enter_csv,
    w_spoken_freq_nlb_with_enter_filtered_csv,
    w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_with_enter_filtered_hr_csv,
    w_spoken_freq_nlb_sumed_csv,
    w_tmp_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
)

p("Import common module")

Import common module


In [81]:
p(w_spoken_freq_nlb_with_enter_csv)
p(w_spoken_freq_nlb_with_enter_filtered_csv)

D:/05.japanproject/jp/freq/w_spoken_freq_nlb_with_enter.csv
D:/05.japanproject/jp/freq/w_spoken_freq_nlb_with_enter_filtered.csv


In [82]:
# 格式化后的的口语词频文件
lst = []
with open(r_spoken_freq_nlb_with_enter_txt, "r", encoding="utf-8") as f:
    content = f.read()  # 一次性读取
    table_matches = pattern_table_spoken.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td_spoken.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "type", "reading", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
    # 将次数映射到新列
    df["count"] = df.groupby(["headword"]).transform("size")
    # value_counts = df["headword"].value_counts()
    # df["count"] = df["headword"].map(value_counts)
    to_csv_sig(df, w_spoken_freq_nlb_with_enter_csv)
p("w_spoken_freq_nlb_with_enter_csv--ok")

w_spoken_freq_nlb_with_enter_csv--ok


In [83]:
df

,headword,type,reading,freq,freq_per,rank,count
0,.,記号,．,"989,922",8.7065,0.99900,1
1,(,記号,（,"851,151",7.4860,0.99890,1
2,*,*,"329,229",2.8956,0.99728,None,265
3,*,記号,＊,"306,861",2.6989,0.99710,265
4,/,記号,／,"199,654",1.7560,0.99524,1
...,...,...,...,...,...,...,...
98173,｝,記号,｝,"3,281",0.0289,0.86545,1
98174,～,記号,～,"152,926",1.3450,0.99366,1
98175,￡,記号,ポンド,82,0.0007,0.46012,1
98176,￣,記号,￣,804,0.0071,0.72208,1


In [84]:
df.dtypes

headword    object
type        object
reading     object
freq        object
freq_per    object
rank        object
count        int64
dtype: object

In [85]:
df.describe()

,count
count,98178.000000
mean,2.215710
std,17.023583
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,265.000000


In [86]:
# 指定列中，只要有一列的数据为空，删除该行,Inplace=True
na_columns = ["headword", "type", "reading", "freq", "rank"]
df.dropna(subset=na_columns, inplace=True)
# 将词频转化为数字
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")
# 将记号和*号相关记录删除
mask = (df["type"] == "記号") | (df["headword"] == "*")
df2 = df[~mask].copy()
df2.sort_values(by="freq", ascending=False, inplace=True)
to_csv_sig(df2, w_spoken_freq_nlb_with_enter_filtered_csv)

p("w_spoken_freq_nlb_with_enter_filtered_csv--ok")

w_spoken_freq_nlb_with_enter_filtered_csv--ok


In [87]:
# 将headword, reading 合成组合字段后， 去重

df2["hr"] = df2.groupby(["headword", "reading"]).transform("size")
# df2["hrt"] = df2["headword"] + "-" + df2["reading"] + "-" + df2["type"]
# hrt_counts = df2["hrt"].value_counts()
# df2["hrt_count"] = df2["hrt"].map(hrt_counts)
# df2["r"] = (
#     df2.groupby("hrt")["rank"]
#     .rank(
#         method="first", ascending=False
#     )  # 并列时按出现顺序排名（类似 SQL 的 ROW_NUMBER）  # 升序排列（默认）
#     .astype(int)
# )
# df[df["r"] == 1].to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
to_csv_sig(df2, w_spoken_freq_nlb_with_enter_filtered_hr_csv)
p(df2)
p("w_spoken_freq_nlb_with_enter_filtered_hr_csv--ok")

      headword   type  reading      freq  freq_per     rank  count  hr
6996         の     助詞        ノ  50267639  442.1104  0.99998      2   2
6674         に     助詞        ニ  35286930  310.3531  0.99996      1   1
9551         を     助詞        ヲ  33707106  296.4584  0.99995      1   1
7123         は     助詞        ハ  31279365  275.1061  0.99994      1   1
2665         が     助詞        ガ  30663103  269.6860  0.99993      2   2
...        ...    ...      ...       ...       ...      ...    ...  ..
98139    Ｔフォール     名詞  ティーフォール         1    0.0000  0.03025      1   1
617     あえぎ求める  動詞-自立  アエギモトメル         1    0.0000  0.07489      1   1
98112      Ｒする  動詞-自立   ラウンドスル         1    0.0000  0.00328      1   1
29336    先入れする  動詞-自立   サキイレスル         1    0.0000  0.04998      1   1
73750    移り広がる  動詞-自立  ウツリヒロガル         1    0.0000  0.06955      1   1

[97450 rows x 8 columns]
w_spoken_freq_nlb_with_enter_filtered_hr_csv--ok


In [89]:
# 处理汇总数据
df_spoken_sumed = df2["freq"].groupby([df["headword"], df["reading"]]).sum().to_frame()
df_spoken_sumed.rename(columns={"freq": "spoken_freq"}, inplace=True)
df_spoken_sumed.sort_values(by="spoken_freq", ascending=False, inplace=True)
to_csv_sig(df_spoken_sumed, w_spoken_freq_nlb_sumed_csv, True)
p(df_spoken_sumed)
p("w_spoken_freq_nlb_sumed_csv--ok")

                  spoken_freq
headword reading             
の        ノ           57221425
に        ニ           35286930
を        ヲ           33707106
は        ハ           31279365
が        ガ           30705135
...                       ...
遡求する     ソキュウスル             1
遠駆け      トオガケ               1
遠近する     オチコチスル             1
ちさする     チサスル               1
咲き賑わう    サキニギワウ             1

[96949 rows x 1 columns]
w_spoken_freq_nlb_sumed_csv--ok


In [91]:
df_spoken_sumed.index

MultiIndex([(     'の',      'ノ'),
            (     'に',      'ニ'),
            (     'を',      'ヲ'),
            (     'は',      'ハ'),
            (     'が',      'ガ'),
            (     'て',      'テ'),
            (     'た',      'タ'),
            (    'ます',     'マス'),
            (     'だ',     'ダロ'),
            (     'と',      'ト'),
            ...
            (  '遠謀する', 'エンボウスル'),
            ('ちびちびする', 'チビチビスル'),
            (  'ちばする',   'チバスル'),
            (   '遠海魚', 'エンカイギョ'),
            (    '遡江',    'ソコウ'),
            (  '遡求する', 'ソキュウスル'),
            (   '遠駆け',   'トオガケ'),
            (  '遠近する', 'オチコチスル'),
            (  'ちさする',   'チサスル'),
            ( '咲き賑わう', 'サキニギワウ')],
           names=['headword', 'reading'], length=96949)